# Amazon Bedrock AgentCore Gateway - Semantic search tutorial

### Tutorial Details


| Information         | Details                                                                          |
|:--------------------|:---------------------------------------------------------------------------------|
| Tutorial type       | Conversational                                                                   |
| Agent type          | Single                                                                           |
| Agentic Framework   | Strands Agents                                                                   |
| LLM model           | Anthropic Claude Sonnet 4                                                        |
| Tutorial components | Creating and using Lambda-backed AgentCore Gateway from Strands Agent            |
| Tutorial vertical   | Cross-vertical                                                                   |
| Example complexity  | Easy                                                                             |
| SDK used            | Amazon BedrockAgentCore Python SDK and boto3  

### Tutorial Architecture
[tbd insert diagram showing Strands Agent calling AgentCore Gateway, with multiple Lambda targets with large numbers of tools, and using Gateway Search]

### Tutorial Key Features

* Creating Amazon Bedrock AgentCore Gateways with Lambda-backed targets
* Using AgentCore Gateway semantic search 
* Using Strands Agents to show how search improves latency

## Prerequisites

To execute this tutorial you will need:
* Python 3.10+
* AWS credentials
* Amazon Bedrock AgentCore SDK
* Strands Agents

## AgentCore Gateway helps solve the challenge of MCP servers that have large numbers of tools
In a typical enterprise setting, agent builders encounter MCP servers that have hundreds or even thousands
of MCP tools. This volume of tools poses challenges for AI Agents, including poor tool selection accuracy, 
increased cost, and higher latency driven by higher token usage from excessive tool metadata.
This can happen when connecting your agents to third party services (e.g., Zendesk, Salesforce,
Slack, JIRA, ...), or to existing enterprise REST services. 
AgentCore Gateway provides a built in semantic search across tools, 
which improves agent latency, cost, and accuracy, while still giving those agents the tools they need. 
Depending on your use case, LLM model, and agent framework, you can see up to 3x better latency by keeping
an agent focused on relevant tools versus providing the full set of hundreds of tools from a typical MCP Server.

## What you will learn in this notebook
In this notebook, we provide a tutorial for AgentCore Gateway search. By the end of this step by step tutorial, you
will understand:

- How to use AgentCore Gateway's built-in search tool to quickly find relevant tools 
- How to integrate tool search results into Strands Agents for improved latency and reduced cost

## Overview of the notebook structure
The notebook is structured with the following sections:

1. Understanding fundamentals of AgentCore Gateway Search
2. Preparing the notebook environment
3. Setting up a Gateway that has hundreds of tools
4. Searching for tools from a Gateway
5. Using Strands Agents with an MCP server that has many tools
6. Adding tool search results to a Strands Agent
7. Showing 3x latency improvement by using tool search

# Understanding fundamentals of AgentCore Gateway Search

When you create an AgentCore Gateway, you have the option to indicate that you want Search enabled.
By default, Gateways are created without Search turned on. For Gateways with search enabled, three things happen:

1. **Vector store is created**. The Gateway service automatically creates a serverless and fully managed vector store for your new Gateway. This will enable a full semantic search across your Gateway tools. 
3. **Vector store is populated**. As you add Gateway Targets to your Gateway, the service will automatically use embeddings behind the scenes to populate the vector store based on all of the tools from the new Target. The tool metadata comes from the JSON defintions of your tools or the OpenAPI Schema specification for your REST services targets.
2. **Search tool (MCP based) is provided**. In addition to all of your user defined tools (from Lambda targets or REST services), the Gateway gets one additional MCP tool that provides semantic search. It is named `x-amz-bedrock-agentcore-search`. The `x-amz-bedrock-agentcore-` prefix ensures there are no name clashes with your user defined tools. We may add more tools like that in the future as well. The search tool has a single argument called `query`. When the search tool is invoked, the Gateway service will do a semantic search using that query, matching it against all available tool metadata (names, descriptions, input schema), and returns the most relevant tools in descending order of relevance.

# Preparing the notebook environment

In [ ]:
# !pip install -r requirements.txt

In [ ]:
## needed for paginated list_tools_sync
# !pip install --upgrade strands-agents@git+https://github.com/strands-agents/sdk-python.git

In [ ]:
# !pip install ~/Downloads/boto3-1.39.4-py3-none-any.whl

In [ ]:
# !pip install ~/Downloads/botocore-1.39.4-py3-none-any.whl

In [ ]:
# !pip install -r requirements.txt 

Import all the required Python libraries, and load environment variables

In [ ]:
from mcp.client.streamable_http import streamablehttp_client
from mcp.types import Tool as MCPTool

from strands import Agent
from strands.models import BedrockModel
from strands.handlers import null_callback_handler
from strands.tools.mcp.mcp_client import MCPClient, MCPAgentTool

import logging
import time
import json
import os
import boto3
import requests

from dotenv import load_dotenv
load_dotenv()


Set up a logger

In [ ]:

# Configure the root strands logger
logging.getLogger("strands").setLevel(logging.ERROR) #INFO) #DEBUG) #

# Add a handler to see the logs
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s", 
    handlers=[logging.StreamHandler()]
)

Check our boto3 version

In [ ]:
boto3.__version__

-- REMOVE -- get the endpoint url for internal testing env

In [ ]:
endpoint_url=os.getenv("genesis_cp_base_url")
endpoint_url

Get our boto3 client for the AgentCore control plane API.

In [ ]:
session = boto3.Session()
agentcore_client = session.client(
    "bedrock-agentcore-control",
    region_name="us-east-1", 
    endpoint_url=endpoint_url, ## this will be removed before we publish
    verify=False
)


--REMOVE-- this is a hack to work around a local env issue. 

In [ ]:
## Workaround for SSL validation issue in my env.
## Remove before publish
import certifi
os.environ['AWS_CA_BUNDLE'] = certifi.where()

# Setting up a Gateway that has hundreds of tools

AgentCore Gateway provides a secure and scalable way to expose a curated set of existing APIs
as MCP tools for your agents. In a production setting, your Gateway resources would be created
using infrastructure as code with tools like CloudFormation, CDK, or Terraform. In this tutorial,
we use the boto3 control plane APIs directly so that you can understand the resources and Gateway APIs.
This will help you more easily get started building and using your own Gateway's, and making more powerful
and secure agents.

At a high level, the steps for setting up your Gateway are:

1. Define what identity providers and credential providers you are using for inbound (agents calling Gateways) and outbound (Gateways calling tools) security, and get [...] needed when creating [...].
2. Create the Gateway using `create_gateway`.
3. Add Gateway Targets using `create_gateway_target`, to expose MCP tools that will be implemented in Lambda or in existing REST services.

In this tutorial, we will use Amazon Cognito as the identity provider (IdP) and will use Lambda-based targets, and AWS IAM for outbound authentication. The same concepts apply when using other IdP's or other target types.

### Creating Amazon Cognito resources

In this tutorial, we assume you have already created resources and have set environment variables for the following:

- your IAM role for Lambda execution (`gateway_lambda_iam_role`)
- a Lambda function for your simple math tools (`calc_lambda_arn`)
- a Lambda function for your restaurant reservation tool (`restaurant_lambda_arn`)
- your Amazon Cognito user pools, giving you a client id (`cognito_client_id`) and a discovery URL (`cognito_discovery_url`)

Here's the JSON tool metadata for the restaurant API. Note that for existing REST services, the API
specs are provided using OpenAPI Schema instead.

In [ ]:
with open('./restaurant/restaurant-api.json') as f:
    data = json.load(f)
print(json.dumps(data, indent=4))

Here are the simple calculator APIs.

In [ ]:
with open('./calc/calc-api.json') as f:
    data = json.load(f)[0:3]
print(json.dumps(data, indent=4))

Here is the Lambda function implementation for the calculator tools.

In [ ]:
from IPython.display import display, Code

file_path = './calc/calc-lambda.py' # Replace with the actual path to your file

with open(file_path, 'r') as f:
    code_content = f.read()

display(Code(code_content, language='python'))

In [ ]:
file_path = './restaurant/restaurant-lambda.py' # Replace with the actual path to your file

with open(file_path, 'r') as f:
    code_content = f.read()

display(Code(code_content, language='python'))

#### Let's create a few helper functions for using the control plane APIs

In [ ]:
def read_apispec(json_file_path):
    try:
        # read json file and return contents as string
        with open(json_file_path, 'r') as file:
            # Parse JSON to Python object
            api_spec = json.load(file)
            return api_spec
            
    except FileNotFoundError:
        return f"Error: File {json_file_path} not found"
    except Exception as e:
        return f"An unexpected error occurred: {str(e)}"

def create_gateway(gateway_name, gateway_desc):
    # Use Cognito for Inbound OAuth to our Gateway
    auth_config = {
        "customJWTAuthorizer": {
            "allowedClients": [os.getenv("cognito_client_id")],
            "discoveryUrl": os.getenv("cognito_discovery_url")
        }
    }

    # Enable semantic search of tools
    search_config = {
        "mcp": {
            "searchType": "SEMANTIC",
            "supportedVersions": ["2025-03-26"]
        }
    }

    # Create the gateway
    response = agentcore_client.create_gateway(
        name=gateway_name,
        roleArn=os.getenv("genesis_iam_role"),
        #kmsKeyArn="arn:aws:kms:us-east-1:966111871240:key/7e526552-383f-435e-a2e7-5d35d49a96c8",
        authorizerType="CUSTOM_JWT",
        description=gateway_desc,
        protocolType="MCP",
        authorizerConfiguration=auth_config,
        protocolConfiguration=search_config
    )

    print(json.dumps(response, indent=2, default=str))
    return response['gatewayId']

def create_gatewaytarget(gateway_id, target_name, target_descr, lambda_arn, api_spec):
    # Add a Lambda target to the gateway 
    response = agentcore_client.create_gateway_target(
        gatewayIdentifier=gateway_id,
        name=target_name,
        description=target_descr,
        targetConfiguration={
            "mcp": {
                "lambda": {
                    "lambdaArn": lambda_arn,
                    "toolSchema": {
                        "inlinePayload": api_spec
                    }
                }
            }
        },
        # Use IAM as credential provider
        credentialProviderConfigurations=[{"credentialProviderType":"GATEWAY_IAM_ROLE"}]
    )
    return response['targetId']

    #print(json.dumps(response, indent=2, default=str))

def list_gateways():
    response = agentcore_client.list_gateways()
    print(json.dumps(response, indent=2, default=str))


In [ ]:
GATEWAY_NAME = "gw-tutorial"

### Creating your first AgentCore Gateway
Now let's create the gateway for this tutorial, providing a name and a description.

In [ ]:
print(f"Create gateway with name: {GATEWAY_NAME}")
gatewayId = create_gateway(gateway_name=GATEWAY_NAME, gateway_desc="AgentCore Gateway Tutorial")
print(f"Gateway created with id: {gatewayId}. Creating gateway target.")

### Adding AgentCore Gateway Targets
In this tutorial, we assume you already have installed a pair of Lambda functions, one for doing simple
math calculations, and another that simulates creating a restaurant reservation. We'll add a Gateway Target
for each of these functions.

Once we've added those targets, we'll add additional targets to simply drive higher MCP tool counts that
help us demonstrate the power of AgentCore Gateway search.

Now that the gateway is created, let's add a target for making restaurant reservations
via a Lambda function.

In [ ]:
restaurant_api_spec = read_apispec("./restaurant/OLD-api.json")
print(restaurant_api_spec)
restaurant_lambda_arn = os.getenv("restaurant_lambda_arn")
print(f"Restaurant Lambda ARN: {restaurant_lambda_arn}")

restaurantTargetId = create_gatewaytarget(gateway_id=gatewayId, 
                                            lambda_arn=restaurant_lambda_arn,
                                            target_name="FoodTools",
                                            target_descr="Restaurant Tools",
                                            api_spec=restaurant_api_spec)
print(f"RestaurantTarget created with id: {restaurantTargetId} on gateway: {gatewayId}")

Here we'll add a second target, this time with a Lambda that implements 4 basic tools (add, subtract,
multiply, divide), and a set of 75 generated tool definitions for investment management (trading, credit research, quantitative analysis, portfolio management). The investment management tool definitions are not 
actually implemented in the Lambda. We are only adding them to get a large volume of tools.

In [ ]:
calc_api_spec = read_apispec("./calc/calc-api.json")
print(f"API spec for calc has {len(calc_api_spec)} functions\n")
calc_lambda_arn = os.getenv("calc_lambda_arn")
print(f"Calc Lambda ARN: {calc_lambda_arn}")

calcTargetId = create_gatewaytarget(gateway_id=gatewayId, 
                                    lambda_arn=calc_lambda_arn,
                                    target_name="CalcTools",
                                    target_descr="Calculation Tools",
                                    api_spec=calc_api_spec)
print(f"CalcTools Target created with id: {calcTargetId} on gateway: {gatewayId}")

To demonstrate the power of gateway search, now add a few more copies of the Calculator target, 
so that we end up with 300+ MCP tools exposed.

In [ ]:
calcTargetId = create_gatewaytarget(gateway_id=gatewayId, 
                                    lambda_arn=calc_lambda_arn,
                                    target_name="Calc2",
                                    target_descr="Calculation 2 Tools",
                                    api_spec=calc_api_spec)
print(f"Calc2 Target created with id: {calcTargetId} on gateway: {gatewayId}")

calcTargetId = create_gatewaytarget(gateway_id=gatewayId, 
                                    lambda_arn=calc_lambda_arn,
                                    target_name="Calc3",
                                    target_descr="Calculation 3 Tools",
                                    api_spec=calc_api_spec)
print(f"Calc3 Target created with id: {calcTargetId} on gateway: {gatewayId}")

calcTargetId = create_gatewaytarget(gateway_id=gatewayId, 
                                    lambda_arn=calc_lambda_arn,
                                    target_name="Calc4",
                                    target_descr="Calculation 4 Tools",
                                    api_spec=calc_api_spec)
print(f"Calc4 Target created with id: {calcTargetId} on gateway: {gatewayId}")



### 

In [ ]:

# use the AgentCore SDK to simplify Cognito setup, and create gw, add target

Let's list the Gateway's that have been created in our account so far.

In [ ]:
resp = agentcore_client.list_gateways()
print(resp['items'])

In [ ]:
GATEWAY_ID = gatewayId #"gw6-gbd0mzhuog"

In [ ]:
resp = agentcore_client.get_gateway(gatewayIdentifier=GATEWAY_ID)
print(resp)

In [ ]:
resp = agentcore_client.list_gateway_targets(gatewayIdentifier=GATEWAY_ID)
print(resp['items'])
targets = resp['items']

In [ ]:
resp = agentcore_client.get_gateway_target(gatewayIdentifier=GATEWAY_ID, targetId=targets[0]['targetId'])
print(resp)

# Searching for tools from a Gateway

### Getting familiar with MCP list tools before we search

Let's define some utility functions to retrieve our MCP endpoint URL for a given Gateway ID, and to 
retrieve our JWT OAuth access token to securely use our Gateway.

In [ ]:
def get_gateway_endpoint(gateway_id):
    response = agentcore_client.get_gateway(
        gatewayIdentifier=gateway_id
    )
    gateway_url = response['gatewayUrl']
    return gateway_url 

def get_oauth_token():
    response = requests.post(
        os.getenv("cognito_token_url"),
        data=f"grant_type=client_credentials&client_id={os.getenv('cognito_client_id')}&client_secret={os.getenv('cognito_client_secret')}&scope={os.getenv('cognito_auth_scope')}",
        headers={'Content-Type': 'application/x-www-form-urlencoded'}
    )
    return response.json()['access_token']

Now that our Gateway is created and has targets, let's grab the MCP URL to that
Gateway. We can retrieve that from the Gateway control plane based on the Gateway ID,
giving back the endpoint URL.

In [ ]:
gatewayEndpoint = get_gateway_endpoint(gateway_id=GATEWAY_ID)
print(f"Gateway Endpoint - MCP URL: {gatewayEndpoint}")

MCP server security is based on OAuth. To interact with our Gateway, we'll need to
retrieve a JWT OAuth access token. 

In [ ]:
jwtToken = get_oauth_token()
print(f"Bearer token: {jwtToken}")

Let's define an `invoke_gateway_tool` utility function. Given an endpoint URL and a JWT token,
you can use this utility to invoke any of the MCP tools that AgentCore Gateway made available
for you when you added Gateway Targets to your Gateway.

In [ ]:
def invoke_gateway_tool(gateway_endpoint, jwt_token, tool_params):
    # print(f"Invoking tool {tool_params['name']}")

    requestBody = {
        "jsonrpc": "2.0",
        "id": 2,
        "method": "tools/call",
        "params": tool_params
    }
    response = requests.post(
        gateway_endpoint,
        json=requestBody,
        headers={'Authorization': f'Bearer {jwt_token}', 'Content-Type': 'application/json'}
    )

    return response.json()

Here's another utility function for using MCP's `tools/list` method for listing the MCP tools
available from your Gateway. Given a Gateway ID and a JWT Token, it retrieves the full set
of tools from that Gateway in agent-ready form. The list contains Strands Agents MCPAgentTool objects
that are suitable for handing your Agent. 

Note that `tools/list` call is paginated, so the function needs to loop, getting a page of
tools at a time, until the `nextCursor` field is no longer populated. The utility function directly
calls the endpoint using HTTPS and the jsonrpc protocol. This is a lower level way to list tools
compared to the `MCPClient` class provided by Strands Agents. We'll see that experience later.

In [ ]:
def get_all_agent_tools_from_mcp_endpoint(gateway_endpoint, jwt_token, client):
    more_tools = True
    tools_count = 0
    tools_list = []

    requestBody = {
        "jsonrpc": "2.0",
        "id": 2,
        "method": "tools/list",
        "params": {}
    }
    next_cursor = ""

    while more_tools:
        if tools_count == 0:
            requestBody["params"] = {}
        else:
            print(f"\nGetting next page of tools since a next cursor was returned\n")
            requestBody["params"] = {
                "cursor": next_cursor
            }

        headers = {'Authorization': f'Bearer {jwt_token}', 'Content-Type': 'application/json'}

        print(f"\n\nListing tools for gateway {gateway_endpoint}")
        # print(f"...requestBody: {requestBody}\n")
        # print(f"...headers: {headers}\n")

        response = requests.post(
            gateway_endpoint, 
            json=requestBody,
            headers=headers
        )

        # print("\nResponse Headers:")
        # for header, value in response.headers.items():
        #     print(f"{header}: {value}")

        tools_json = response.json()
        tools_count += len(tools_json["result"]["tools"])

        for tool in tools_json["result"]["tools"]:
            mcp_tool = MCPTool(name=tool['name'],
                            description=tool['description'],
                            inputSchema=tool['inputSchema'])
            mcp_agent_tool = MCPAgentTool(mcp_tool, client)
            short_descr = tool['description'][0:40] + '...'
            print(f"adding tool '{mcp_agent_tool.tool_name}' - {short_descr}")
            tools_list.append(mcp_agent_tool)

        if "nextCursor" in tools_json["result"]:
            next_cursor = tools_json["result"]["nextCursor"]
            more_tools = True
        else:
            more_tools = False

    print(f"\nTotal tools found: {tools_count}\n")
    return tools_list

Let's call it and see what we get.

In [ ]:
client = MCPClient(lambda: streamablehttp_client(f"{gatewayEndpoint}",
                                                    headers={"Authorization": f"Bearer {jwtToken}"}))
with client:
    all_tools = get_all_agent_tools_from_mcp_endpoint(gateway_endpoint=gatewayEndpoint, 
                                            jwt_token=jwtToken, client=client)
    print(f"\nFound {len(all_tools)} tools from list_tools_sync() on mcp client\n")

If you have written any Python based MCP client, you are likely familiar the `list_tools_sync()` method 
which givest you back the set of tools available from the MCP Server which the client is associated
with. But did you know MCP list tools is also paginated? By default, you will only get the first small
subset of tools returned. For simple MCP servers, you may not have
noticed, but for many real world MCP servers, your code needs to loop, grabbing pages of tools at a time
until there are no more pages. The following utility `get_all_mcp_tools` does just that. It returns
the full list of tools from a given Strands Agent MCP Client.

In [ ]:
def get_all_mcp_tools(client):
    more_tools = True
    tools = []
    pagination_token = None
    while more_tools:
        tmp_tools = client.list_tools_sync(pagination_token=pagination_token)
        tools.extend(tmp_tools)
        if tmp_tools.pagination_token is None:
            more_tools = False
        else:
            more_tools = True 
            pagination_token = tmp_tools.pagination_token
    return tools


Let's give it a try with our Gateway and find out how many tools the Python client finds. 
First we'll create an MCPClient object based on our endpoint URL and our JWT bearer token.

In [ ]:
client = MCPClient(lambda: streamablehttp_client(f"{gatewayEndpoint}",
                                                    headers={"Authorization": f"Bearer {jwtToken}"}))


Now we retrieve the full set of tools across many pages of tools returned by the MCP server.

In [ ]:
with client:
    all_tools = get_all_mcp_tools(client)
    print(f"\nFound {len(all_tools)} tools from list_tools_sync() on mcp client\n")

We have now seen 2 different ways to get the full set of tools from your Gateway using it
as an MCP Server: 1/ directly using jsonrpc, 2/ using the `list_tools_sync()` method on the
Strands Agent MCPClient. For typical developers building agents, you'll be using the latter.

### Using the Gateway search tool
Now lets try our first semantic search on the Gateway using its built-in search tool.

First let's define a simple utility function to execute the search tool using MCP.
Just like for listing tools, we need the gateway endpoint and JWT token. Other than that,
all we need to pass in is the search query. The Gateway search tool will do the rest,
matching that query against the serverless vector store that it automatically manages on your behalf.

In [ ]:
def tool_search(gateway_endpoint, jwt_token, query):
    toolParams = {
        "name": "x-amz-bedrock-agentcore-search",
        "arguments": {
            "query": query
        }
    }
    toolResp = invoke_gateway_tool(gateway_endpoint=gateway_endpoint, 
                                   jwt_token=jwt_token, 
                                   tool_params=toolParams)
    tools = toolResp["result"]["structuredContent"]["tools"]
    return tools

In [ ]:
start_time = time.time()
tools_found = tool_search(gateway_endpoint=gatewayEndpoint,
                                jwt_token=jwtToken,
                                query="find me 3 credit research tools")
end_time = time.time()
print(f"tool search via direct Gateway invocation took {end_time - start_time} seconds")
print(f"{len(tools_found)} tools were found.")
print(f"Top tool: {tools_found[0]['name']}")

Notice how fast the search returns, in under a second in most cases. The results are returned
in descending order of search relevance based on matching the query to the tool metadata. So,
the most relevant tools are first on the list. The intial implementation of search gives back
up to 10 results. You could then use all of these tools in your agent, or simply pick the first one
or two.

# Using Strands Agents with an MCP server that has many tools

In [ ]:
# !pip install -U --force-reinstall certifi

In [ ]:
# import httpx
# def create_insecure_http_client(**kwargs) -> httpx.AsyncClient:
#     """Create an HTTPX client with SSL verification disabled."""
#     kwargs['verify'] = False
#     # Optionally add other configurations
#     kwargs.setdefault('timeout', httpx.Timeout(30.0))
#     return httpx.AsyncClient(**kwargs)
# jwtToken = get_oauth_token()
# client = MCPClient(lambda: streamablehttp_client(f"{gatewayEndpoint}",
#                                                  headers={"Authorization": f"Bearer {jwtToken}"}, 
#                                                  httpx_client_factory=create_insecure_http_client))

In [ ]:
import sys 
# sys.path.append("./PyAmazonCACerts/src")

In [ ]:
# import amazoncerts 

In [ ]:
import certifi
os.environ['AWS_CA_BUNDLE'] = certifi.where()

In [ ]:
os.getenv('AWS_CA_BUNDLE')

Here we select a model to use with our Strands Agent. 
For this notebook, we are using Amazon Bedrock models, but Strands and AgentCore
can work with any LLM.

In [ ]:
# from botocore.config import Config as BotocoreConfig
bedrockmodel = BedrockModel(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    temperature=0.7,
    streaming=True,
    boto_session=session
)

In [ ]:
jwtToken = get_oauth_token()
client = MCPClient(lambda: streamablehttp_client(f"{gatewayEndpoint}",
                                                    headers={"Authorization": f"Bearer {jwtToken}"}))
with client:
    all_tools = get_all_mcp_tools(client)
    print(f"\nFound {len(all_tools)} tools from list_tools_sync() on mcp client\n")
    simple_agent = Agent(model=bedrockmodel, tools=all_tools, callback_handler=null_callback_handler)
    print(simple_agent.tool_names)
    math_input = "add 100 plus 50"
    result = simple_agent(math_input)
    print(f"{result.message['content'][0]['text']}")


In [ ]:
jwtToken = get_oauth_token()
client = MCPClient(lambda: streamablehttp_client(f"{gatewayEndpoint}",
                                                    headers={"Authorization": f"Bearer {jwtToken}"}))
with client:
    all_tools = get_all_mcp_tools(client)
    print(f"\nFound {len(all_tools)} tools from list_tools_sync() on mcp client\n")
    simple_agent = Agent(model=bedrockmodel, tools=all_tools, callback_handler=null_callback_handler)
    direct_result = simple_agent.tool.Calc2___add_numbers(firstNumber=10, secondNumber=20)
    resp_json = json.loads(direct_result['content'][0]['text'])
    print(f"direct result = {resp_json['response']['payload']}")


Let's show a simple example of a Strands Agents that uses the full list of tools
from an MCP Server. 

# Adding tool search results to a Strands Agent

Now let's look at how the returned tools from a search can be added to your
Strands Agent. To make the coding simpler, let's provide a utility function that
maps tool search results to Strands MCPAgentTool objects. Simply pass in the 
search results, and indicate how many of those results you want to pass to your
agent.

In [ ]:
def tools_to_strands_mcp_tools(tools, top_n):
    strands_mcp_tools = []
    for tool in tools[:top_n]:
        mcp_tool = MCPTool(name=tool['name'],
                        description=tool['description'],
                        inputSchema=tool['inputSchema'])
        strands_mcp_tools.append(MCPAgentTool(mcp_tool, client))
    return strands_mcp_tools

# Showing 3x latency improvement by using tool search

Now that we know how to use our Gateway tools from Strands, and we know how to search for tools and
add them to an agent, lets show the power of search by highlighting the significant latency reduction
and input token usage that can be delivered.

To demonstrate that, we compare side by side 2 approaches:

1. Without search. We add the full set of MCP tools that the MCP server exposes (300+ in our case) to our agent and let the agent do its tool selection and invocation accordingly.
2. Using search. In the second approach, we do a search based on the topic at hand, and only send in the most relevant tools to the agent.

To normalize the latency distribution and get a meaningful comparison, we perform multiple iterations of
each approach. Also, to avoid over-stating the gains, when doing the search approach, we include not only the
latency of the agent invocation, but also the latency of performing the up front search as well. For each 
iteration, we hand the agent a math task (add 2 numbers) and a food task (book a restaurant reservation).

The results below highlight the potential gains of 3x latency reduction, and even greater reduction in
input token usage. The token usage savings translate to cost savings, but note that for many model providers,
input tokens are much lest costly. Even so, for a large scale agent deployment, even input token usage costs
can add up, so dynamic search can benefit agent runtime costs as well.

In [ ]:
jwtToken = get_oauth_token()
client = MCPClient(lambda: streamablehttp_client(f"{gatewayEndpoint}",
                                                    headers={"Authorization": f"Bearer {jwtToken}"}))

In [ ]:
iterations = 2

with client:
    all_tools = get_all_mcp_tools(client)
    print(f"\nFound {len(all_tools)} tools from list_tools_sync() on mcp client\n")
    heavy_agent = Agent(model=bedrockmodel, tools=all_tools, callback_handler=null_callback_handler)

    math_input = "add 100 plus <iteration>"
    food_input = "book me a table for 2 at Burger King under name Jo Smith at 7pm August <day>"

    print("using agent with ALL tools...")
    start_time = time.time()
    for i in range(iterations):
        result = heavy_agent(math_input.replace("<iteration>", str(i+1)))
        print(f"{i+1}) {result.message['content'][0]['text']}")

        result = heavy_agent(food_input.replace("<day>", str(i+1)))
        print(f"{i+1}) {result.message['content'][0]['text']}")
    end_time = time.time()
    full_tokens = result.metrics.accumulated_usage['totalTokens']
    elapsed_time = end_time - start_time
    print(f"\nTotal time: {elapsed_time:.1f} s, tokens: {full_tokens:,d}\n")

    print("using agent with ONLY tools from focused search...")
    start_time = time.time()
    messages = []
    light_agent = Agent()
    for i in range(iterations):
        print(f"Searching for an ADDING tool from endpoint with full set of tools...")
        tools_found = tool_search(gateway_endpoint=gatewayEndpoint,
                                jwt_token=jwtToken,
                                query="tools for simply adding two numbers")
        print(f"Top tool found: {tools_found[0]['name']}\n")
        light_agent = Agent(model=bedrockmodel, tools=tools_to_strands_mcp_tools(tools_found, 1), 
                            messages=messages, callback_handler=null_callback_handler)
        light_result = light_agent(math_input.replace("<iteration>", str(i+1)))
        print(f"{i+1}) {light_result.message['content'][0]['text']}")
        messages = light_agent.messages
        light_tokens = light_result.metrics.accumulated_usage['totalTokens']

        print(f"Searching for a RESTAURANT BOOKING tool from endpoint with full set of tools...")
        tools_found = tool_search(gateway_endpoint=gatewayEndpoint,
                                jwt_token=jwtToken,
                                query="tools for booking a restaurant reservation")
        print(f"Top tool found: {tools_found[0]['name']}\n")
        light_agent = Agent(model=bedrockmodel, tools=tools_to_strands_mcp_tools(tools_found, 1), 
                            messages=messages, callback_handler=null_callback_handler)
        light_result = light_agent(food_input.replace("<day>", str(i+1)))
        print(f"{i+1}) {light_result.message['content'][0]['text']}")
        messages = light_agent.messages
        light_tokens = light_result.metrics.accumulated_usage['totalTokens']
    end_time = time.time()
    light_elapsed_time = end_time - start_time
    print(f"\nTotal time: {light_elapsed_time:.1f} s, tokens: {light_tokens:,d}\n")

    print(f"\n\nLatency without search: {elapsed_time:.1f}s, using search: {light_elapsed_time:.1f}s")
    print(f"Tokens without search: {full_tokens:,d}, using search: {light_tokens:,d}")


# Cleaning up resources

First let's define some helper functions for cleaning up AgentCore Gateway resources.

In [ ]:
def delete_gatewaytarget(gateway_id):
    response = agentcore_client.list_gateway_targets(
        gatewayIdentifier=gateway_id
    )
    
    print(f"Found {len(response['items'])} targets for the gateway")

    for target in response['items']:
        print(f"Deleting target with Name: {target['name']} and Id: {target['targetId']}")

        response = agentcore_client.delete_gateway_target(
            gatewayIdentifier=gateway_id,
            targetId=target['targetId']
        )

def delete_gateway(gateway_id):
    response = agentcore_client.delete_gateway(
        gatewayIdentifier=gateway_id
    )


### Deleting Gateway Targets

In [ ]:
delete_gatewaytarget(gateway_id=GATEWAY_ID)

### Deleting the Gateway itself

In [ ]:
delete_gateway(gateway_id=GATEWAY_ID)

### [Optional] Delete the Lambda functions